In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pickle
import os
import constants_aspects_analysis as constants
import preprocess_helper as helper
import re
from xml import etree
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize,sent_tokenize, pos_tag
from nltk.corpus import sentiwordnet as swn
from data_logger import DataLogger

In [2]:
def find_listxml(acorpus, aname_tag):
    return acorpus.findall(aname_tag)

def string_nested_xml(axml):
    return ' '.join([the_aiter for the_aiter in axml.itertext()])

def get_firstchild(axml):
    try:
        if len(axml.getchildren()) > 0:
            return axml.getchildren()[0].tag
        else:
            raise (Exception('ListIndex', 'aXmlElement input has no children.'))
    except Exception as e:
        print (str(e))

def xml_unique_valid(axml, alist_tag_allowed):
    return (len(axml.getchildren()) == 0) or (get_firstchild(axml) in alist_tag_allowed)

def get_listsentence_unique(alist_xml, alist_tag_allowed):
    the_listsentence = []
    for the_axml in alist_xml:
        if xml_unique_valid(the_axml, alist_tag_allowed):
            the_listsentence.append(string_nested_xml(the_axml))

    return the_listsentence

def xml_name_valid(axml, atag_name):
    return axml.tag == atag_name

def get_listxml_child(alist_xml, atag_name):
    the_listreturn = []
    for the_axml in alist_xml:
        for the_achild in the_axml:
            if xml_name_valid(the_achild, atag_name):
                the_listreturn.append(the_achild)

    return the_listreturn

In [3]:
data_logger_util = DataLogger()
helper.variable_information(constants.const, constants.const.PRINT_STATUS)

{'CORPUS_XML_FILE_LOCATION': './data/User Review Structure Analysis '
                             '(URSA)/Classified_Corpus.xml',
 'FILE_LOG_AMBIENCE_NOT_PASS': 'ambience_not_pass.txt',
 'FILE_LOG_AMBIENCE_NOT_PASS_LOCATION': './data/User Review Structure Analysis '
                                        '(URSA)/ambience_not_pass.txt',
 'FILE_LOG_FOOD_NOT_PASS': 'food_not_pass.txt',
 'FILE_LOG_FOOD_NOT_PASS_LOCATION': './data/User Review Structure Analysis '
                                    '(URSA)/food_not_pass.txt',
 'FILE_LOG_STAFF_NOT_PASS': 'staff_not_pass.txt',
 'FILE_LOG_STAFF_NOT_PASS_LOCATION': './data/User Review Structure Analysis '
                                     '(URSA)/staff_not_pass.txt',
 'FILE_SEED_WORD': 'seed_words.txt',
 'FILE_SEED_WORD_LOCATION': './data/User Review Structure Analysis '
                            '(URSA)/seed_words.txt',
 'FOLDER_DATASET': './data/User Review Structure Analysis (URSA)/',
 'LABEL_REVIEW_AMBIENCE': 2,
 'LABEL_REVIEW_FOOD':

In [4]:
corpus_tree = ET.parse(constants.const.CORPUS_XML_FILE_LOCATION)
corpus = corpus_tree.getroot()

In [5]:
listdocument = find_listxml(corpus, constants.const.TAG_XML_REVIEW)

In [6]:
listxml_child_food = get_listxml_child(listdocument, constants.const.TAG_NAME_FOOD)
listxml_child_staff = get_listxml_child(listdocument, constants.const.TAG_NAME_STAFF)
listxml_child_ambience = get_listxml_child(listdocument, constants.const.TAG_NAME_AMBIENCE)

In [7]:
helper.print_length_variables([listxml_child_food, listxml_child_staff, listxml_child_ambience], \
                               ['listxml_child_food', 'listxml_child_staff', 'listxml_child_ambience'], \
                               constants.const.PRINT_STATUS)

Length  listxml_child_food 	:  96235
Length  listxml_child_staff 	:  32892
Length  listxml_child_ambience 	:  16803


In [8]:
##### FOOD ##### STAFF ##### AMBIENCE
listxml_unique_food = get_listsentence_unique(listxml_child_food, \
                                                constants.const.TAG_NAME_POLARITY_ALLOWED)
listxml_unique_staff = get_listsentence_unique(listxml_child_staff, \
                                              constants.const.TAG_NAME_POLARITY_ALLOWED)
listxml_unique_ambience = get_listsentence_unique(listxml_child_ambience, \
                                              constants.const.TAG_NAME_POLARITY_ALLOWED)

In [9]:
helper.print_length_variables([listxml_unique_food, listxml_unique_staff, listxml_unique_ambience], \
                              ['listxml_unique_food', 'listxml_unique_staff', 'listxml_unique_ambience'], \
                              constants.const.PRINT_STATUS)

Length  listxml_unique_food 	:  62348
Length  listxml_unique_staff 	:  23730
Length  listxml_unique_ambience 	:  13385


In [13]:
listprocessed_food = helper.process_listtext(listxml_unique_food, constants.const.LENGTH_FOOD_ALLOWED, \
                                             constants.const.FILE_LOG_FOOD_NOT_PASS_LOCATION)
listprocessed_staff = helper.process_listtext(listxml_unique_staff, constants.const.LENGTH_STAFF_ALLOWED,\
                                              constants.const.FILE_LOG_STAFF_NOT_PASS_LOCATION)
listprocessed_ambience = helper.process_listtext(listxml_unique_ambience, constants.const.LENGTH_AMBIENCE_ALLOWED, \
                                                 constants.const.FILE_LOG_AMBIENCE_NOT_PASS_LOCATION)

In [14]:
helper.print_length_variables([listprocessed_food, listprocessed_staff, listprocessed_ambience], \
                              ['listprocessed_food', 'listprocessed_staff', 'listprocessed_ambience'], \
                              constants.const.PRINT_STATUS)

Length  listprocessed_food 	:  62348
Length  listprocessed_staff 	:  23730
Length  listprocessed_ambience 	:  13385


In [15]:
listlabeled_food = helper.label_listdata(listprocessed_food, constants.const.LABEL_REVIEW_FOOD)
listlabeled_staff = helper.label_listdata(listprocessed_staff, constants.const.LABEL_REVIEW_STAFF)
listlabeled_ambience = helper.label_listdata(listprocessed_ambience, constants.const.LABEL_REVIEW_AMBIENCE)

In [16]:
helper.print_length_variables([listlabeled_food, listlabeled_staff, listlabeled_ambience], \
                              ['listlabeled_food', 'listlabeled_staff', 'listlabeled_ambience'], \
                              constants.const.PRINT_STATUS)

helper.print_listsample([listxml_unique_ambience, listprocessed_ambience, listlabeled_ambience], \
                        ['listxml_unique_ambience', 'listprocessed_ambience', 'listlabeled_ambience'], \
                        constants.const.SAMPLE_INDEX_FROM, constants.const.SAMPLE_INDEX_TO, constants.const.PRINT_STATUS)

Length  listlabeled_food 	:  62348
Length  listlabeled_staff 	:  23730
Length  listlabeled_ambience 	:  13385
###  listxml_unique_ambience ###
#  In Short In nice weather , the front doors are flung open to the air ; cold nights find patrons tucked into rear booths . 
#  Great for groups , great for a date , great for early brunch or a nightcap . 
#  Upon using their bathroom ( btw , don't ) I found that the other side of the restaurant was air condtioned and there were a few empty tables . 
#  It became impossible to stand and have a drink or any type of conversation . 
#  its a fun environment and they play good music as well . 
###  listprocessed_ambience ###
# ['short', 'nice', 'weather', 'front', 'door', 'flung', 'open', 'air', 'cold', 'night', 'find', 'patron', 'tuck', 'rear', 'booth']
# ['great', 'group', 'great', 'date', 'great', 'earli', 'brunch', 'nightcap']
# ['upon', 'use', 'bathroom', 'btw', 'found', 'side', 'restaur', 'air', 'condtion', 'empti', 'tabl']
# ['becam', 'impos

In [17]:
vocab, vocab_dict = helper.create_vocab_listsentence(listprocessed_food + listprocessed_staff + listprocessed_ambience, constants.const.MIN_FREQ_ALLOWED)

list_onehot = helper.create_listonehot(listlabeled_food + listlabeled_staff + listlabeled_ambience, \
                                       vocab, vocab_dict)

# Save Into File
np.save (constants.const.TRAIN_FILE_LOCATION, list_onehot)
data_logger_util.pickle(constants.const.VOCAB_FILE_LOCATION, vocab_dict)

In [28]:
# variable create corpus and compute coherent
folder_corpus="corpus"
filename_corpus="ursa_a"
filename_corpus0="ursa.0"
filename_corpus1="ursa.1"
filename_corpus2="ursa.2"

In [32]:
# create corpus
file_corpus = open('corpus/{0}'.format(filename_corpus), 'w+')
file_corpus0 = open('corpus/{0}'.format(filename_corpus0), 'w+')
file_corpus1 = open('corpus/{0}'.format(filename_corpus1), 'w+')
file_corpus2 = open('corpus/{0}'.format(filename_corpus2), 'w+')

for sentence in listprocessed_food:
    file_corpus0.write('{0}\n'.format(" ".join([w for w in sentence])))
    file_corpus.write('{0}\n'.format(" ".join([w for w in sentence])))
    
for sentence in listprocessed_staff:
    file_corpus1.write('{0}\n'.format(" ".join([w for w in sentence])))
    file_corpus.write('{0}\n'.format(" ".join([w for w in sentence])))
    
for sentence in listprocessed_ambience:  
    file_corpus2.write('{0}\n'.format(" ".join([w for w in sentence])))
    file_corpus.write('{0}\n'.format(" ".join([w for w in sentence])))

file_corpus.close()
file_corpus0.close()
file_corpus1.close()
file_corpus2.close()